In [32]:
import nucle
# Path to the file you want to process
file_path_conll14st = 'release3.3/data/conll14st-preprocessed.m2'  # Update this with the correct path to your file
file_path_bea2019 = 'release3.3/bea2019/nucle.train.gold.bea19.m2'
file_path_test = 'benchmark_data/noalt/official-2014.0.m2'
file_path_a1 = '10gec_annotations/A1.m2'
file_path_a2 = '10gec_annotations/A2.m2'
file_path_a3 = '10gec_annotations/A3.m2'
file_path_a4 = '10gec_annotations/A4.m2'
file_path_a5 = '10gec_annotations/A5.m2'
file_path_a6 = '10gec_annotations/A6.m2'
file_path_a7 = '10gec_annotations/A7.m2'
file_path_a8 = '10gec_annotations/A8.m2'
file_path_a9 = '10gec_annotations/A9.m2'
file_path_a10 = '10gec_annotations/A10.m2'

# Generate examples from the file
nuc = nucle.NUCLE()
examples_conll14st = list(nuc._generate_examples(file_path_conll14st))
examples_bea2019 = list(nuc._generate_examples(file_path_bea2019))
examples_test = list(nuc._generate_examples(file_path_test))
examples_a1 = list(nuc._generate_examples(file_path_a1))
examples_a2 = list(nuc._generate_examples(file_path_a2))
examples_a3 = list(nuc._generate_examples(file_path_a3))
examples_a4 = list(nuc._generate_examples(file_path_a4))
examples_a5 = list(nuc._generate_examples(file_path_a5))
examples_a6 = list(nuc._generate_examples(file_path_a6))
examples_a7 = list(nuc._generate_examples(file_path_a7))
examples_a8 = list(nuc._generate_examples(file_path_a8))
examples_a9 = list(nuc._generate_examples(file_path_a9))
examples_a10 = list(nuc._generate_examples(file_path_a10))

Repo card metadata block was not found. Setting CardData to empty.


In [33]:
import re

import re

def detokenize(tokens):
    sentence = ""
    open_double_quote = True
    open_single_quote = True
    no_space_tokens = ["'s", "n't", "'S", "'m", "'ll", "'ve", "'d", "'re", "--", ".-", "%"]
    prev_no_space_tokens = ["'", "$", "-", "'", '"', ' ', '£', '(', '[', "{"]
    
    for token in tokens:
        if token[-1] == '"' and token != '"':
            open_double_quote = not open_double_quote
        if token[-1] == "'" and token != "'":
            open_single_quote = not open_single_quote
        if token == '"':
            if open_double_quote:
                sentence += ' "' if sentence and not sentence[-1].isspace() else '"'
                open_double_quote = False
            else:
                sentence = sentence.rstrip() + '" '
                open_double_quote = True
        elif token == "'":         
            if open_single_quote:
                sentence += " '" if sentence and not sentence[-1].isspace() else "'"
                open_single_quote = False
            else:
                sentence = sentence.rstrip() + "' "
                open_single_quote = True
        elif token in {".", ",", "!", "?", ":", ";", "-"}:
            if sentence and sentence[-1] == ' ':
                sentence = sentence[:-1]
            sentence += token
        elif token in {")", "]", "}"}:
            sentence = sentence.rstrip() + token
        elif token in {"(", "[", "{"}:
            sentence += token
        elif token not in no_space_tokens and token[0] == "'" and not (len(token) == 3 and token[1:].isdigit()):
            if open_single_quote:
                sentence += " "
                sentence += token
                open_single_quote = False
            else:
                sentence += token[0] + ' ' + token[0:]
                open_single_quote = True
        elif token not in no_space_tokens and token[0] == ".":
            sentence += token[0] + ' ' + token[0:]
        elif token not in no_space_tokens and token[0] == ",":
            sentence += token[0] + ' ' + token[0:]
        else:
            if sentence and sentence[-1] not in prev_no_space_tokens and token not in no_space_tokens:
                sentence += " "
            sentence += token
    return sentence

# Example usage
tokens = ['Molten-Salt-Reactor', 'Technology', 'Gaps', '(', 'p.4', ')', 'Retrieved', 'October', '13', ',', '2009', 'from', 'Proceedings', 'of', 'ICAPP', "'06", ',', 'Reno', ',', 'NV', 'USA', ',', 'Paper', '6295']
sentence = detokenize(tokens)
print(sentence)

Molten-Salt-Reactor Technology Gaps(p.4) Retrieved October 13, 2009 from Proceedings of ICAPP '06, Reno, NV USA, Paper 6295


In [35]:
import pandas as pd
all_ds = [examples_conll14st, examples_bea2019, examples_test, examples_a1, examples_a2, examples_a3, examples_a4, examples_a5, examples_a6, examples_a7, examples_a8, examples_a9, examples_a10]
all_names = ["examples_conll14st", "examples_bea2019", "examples_test", "examples_a1", "examples_a2", "examples_a3", "examples_a4", "examples_a5", "examples_a6", "examples_a7", "examples_a8", "examples_a9", "examples_a10"]
tokens = set()
for ds, name in zip(all_ds, all_names):
    before_statements = []
    after_statements = []
    corrections = []
    for indexed_example in ds:
        example = indexed_example[1]
        before = example['src_tokens']
        after = example['tgt_tokens']
        correct = example['corrections']
        before_statements.append(detokenize(before))
        after_statements.append(detokenize(after))
        corrections.append(correct)
    data = {
        "Before Statements": before_statements,
        "After Statements": after_statements,
        "Error Correction": corrections
    }
    df = pd.DataFrame(data)
    df.to_csv(f"{name}.csv", index = False)